In [135]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [136]:
#netflix_training = pd.read_csv("/kaggle/input/stockdataset/Stock-Dataset/Training/NFLX.csv")
#netflix_testing = pd.read_csv("/kaggle/input/stockdataset/Stock-Dataset/Testing/NFLX.csv")
netflix_training = pd.read_csv("/kaggle/input/stockdataset/Stock-Dataset/Training/NFLX.csv")
netflix_testing = pd.read_csv("/kaggle/input/stockdataset/Stock-Dataset/Testing/NFLX.csv")


In [137]:
netflix_training

In [138]:
netflix_training.isna().sum()

In [139]:
netflix_testing

In [140]:
netflix_testing.isna().sum()

In [141]:
netflix_training["Date"] = netflix_training["Date"].astype("datetime64")
netflix_training =  netflix_training.set_index("Date")

In [142]:
netflix_training

In [143]:
netflix_testing["Date"] = netflix_testing["Date"].astype("datetime64")
netflix_testing =  netflix_testing.set_index("Date")

In [144]:
netflix_testing

In [145]:
netflix_training.shape, netflix_testing.shape

In [146]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))

plt.plot( netflix_training["Close"],color = 'green')
plt.plot( netflix_testing["Close"],color = 'red')
plt.title("NETFLIX")
plt.xlabel("Date")
plt.ylabel("Close Values")
plt.show()

In [147]:
df_training = netflix_training["Close"]
df_testing = netflix_testing["Close"]

In [148]:
df_training.shape, df_testing.shape

In [149]:
df_training

In [150]:
df_training = pd.Series(df_training, index=df_training.index)
df_testing = pd.Series(df_testing, index=df_testing.index)


In [151]:
df_training

In [152]:
def SMAPE(y,yhat):
    y, yhat = np.array(y), np.array(yhat)
    new = (np.abs(yhat) + np.abs(y))
    smap=2*np.abs(y-yhat)/new
    smap_final = np.mean(smap)
    return 100*smap_final

def RMSE(y, yhat):
    y, yhat = np.array(y), np.array(yhat)
    return np.sqrt(np.mean(np.square(y - yhat)))

def MAE(y,yhat):
    y, yhat = np.array(y), np.array(yhat)
    mae = np.mean(np.abs(y - yhat))
    return mae

def msMAPE(y,yhat):
    epsilon = 0.1  
    y, yhat = np.array(y), np.array(yhat)
    comparator = np.zeros((y.shape[0],))+epsilon
    sum_1 = np.maximum(comparator, (abs(y) + abs(yhat) + epsilon))
    msmape = 2 * abs(y -yhat ) / (sum_1)
    return 100*np.mean(msmape)

def MASE(training_series, testing_series, prediction_series,S):
    y_train = np.array(training_series)
    y_test = np.array(testing_series)
    yhat = np.array(prediction_series)
    n1 = y_train.shape[0]
    d = (np.abs(np.diff(y_train,n=S)).sum())/(n1-S)
    errors = np.abs(y_test - yhat )
    return np.mean(errors)/d

# SES

In [153]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing

fit1 = SimpleExpSmoothing(df_training).fit(optimized=True)
fcast1 = fit1.forecast(len(df_testing)).rename(r'$\alpha=%s$'%fit1.model.params['smoothing_level'])
ses_smape = SMAPE(df_testing,fcast1)
ses_rmse = RMSE(df_testing,fcast1)
ses_mae = MAE(df_testing,fcast1)
ses_msmape = msMAPE(df_testing,fcast1)
ses_mase = MASE(df_training,df_testing,fcast1,7)


In [154]:
fit1.summary()

In [155]:
plt.figure(figsize=(16,8))

pred=pd.Series(fcast1) 
pred.index = df_testing.index
fcast1.plot(color='green', legend=True)

plt.plot(fit1.fittedvalues,color='green')
plt.plot(df_testing,color='red')
plt.plot(pred,color='blue')
plt.title("NETFLIX")
plt.xlabel("Date")
plt.ylabel("Close Values")

In [156]:
plt.figure(figsize=(16,8))

pred=pd.Series(fcast1) 
pred.index = df_testing.index
plt.plot(fit1.fittedvalues,color='green')
plt.plot(df_testing,color='red')
plt.plot(pred,color='blue')
plt.title("NETFLIX")
plt.xlabel("Date")
plt.ylabel("Close Values")

In [157]:
def get_error_metrics(model, mase, smape, rmse, mae, msmape):
    print(model + " is used, the following are result Obtained:")
    print("The mase is: " + str(mase))
    print("The smape is: " + str(smape))
    print("The rmse is: " + str(rmse))
    print("The mae is: " + str(mae))
    print("The msmape is: " + str(msmape))


In [158]:
get_error_metrics("SES", ses_mase, ses_smape, ses_rmse, ses_mae, ses_msmape)

# ETS

In [159]:
from statsmodels.tsa.seasonal import seasonal_decompose
final = seasonal_decompose(df_training,model='additive', period = 7)

In [160]:
fig = final.plot()


In [161]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel 
#train_y_series = pd.Series(train_y_list[0]).astype('float64')

ets_model = ETSModel(df_training, error='mul', trend='mul', seasonal = 'add', seasonal_periods=7)
ets_fitted = ets_model.fit()

In [162]:
print(ets_fitted.summary())


In [163]:
ets_fcast1 = ets_fitted.forecast(len(df_testing))


In [164]:
ets_smape = SMAPE(df_testing,ets_fcast1)
ets_rmse = RMSE(df_testing,ets_fcast1)
ets_mae = MAE(df_testing,ets_fcast1)
ets_msmape = msMAPE(df_testing,ets_fcast1)
ets_mase = MASE(df_training,df_testing,ets_fcast1,7)

In [165]:
get_error_metrics("ETS", ets_mase, ets_smape, ets_rmse, ets_mae, ets_msmape)

# THETA

In [166]:
from statsmodels.tsa.forecasting.theta import ThetaModel

tm = ThetaModel(df_training, period = 7)
res = tm.fit()
theta_fcast1 = res.forecast(len(df_testing))
theta_smape = SMAPE(df_testing,theta_fcast1)
theta_rmse = RMSE(df_testing,theta_fcast1)
theta_mae = MAE(df_testing,theta_fcast1)
theta_msmape = msMAPE(df_testing,theta_fcast1)
theta_mase = MASE(df_training,df_testing,theta_fcast1,7)


In [167]:
get_error_metrics("THETA", theta_mase, theta_smape, theta_rmse, theta_mae, theta_msmape)

# TBATS

In [168]:
pip install tbats


In [169]:
from tbats import TBATS, BATS
estimator = TBATS()
model = estimator.fit(df_training)
y_forecast = model.forecast(len(df_testing))

tbats_smape = SMAPE(df_testing,y_forecast)
tbats_rmse = RMSE(df_testing,y_forecast)
tbats_mae = MAE(df_testing,y_forecast)
tbats_msmape = msMAPE(df_testing,y_forecast)
tbats_mase = MASE(df_training,df_testing,y_forecast,7)


In [170]:
get_error_metrics("TBATS", tbats_mase, tbats_smape, tbats_rmse, tbats_mae, tbats_msmape)

# ARIMA

In [171]:
from statsmodels.tsa.arima.model import ARIMA
   
fit1=ARIMA(df_training,order=(10,1,1)).fit()
arima_fcast1 = fit1.forecast(len(df_testing))
arima_smape = SMAPE(df_testing,arima_fcast1)
arima_rmse = RMSE(df_testing,arima_fcast1)
arima_mae = MAE(df_testing,arima_fcast1)
arima_msmape = msMAPE(df_testing,arima_fcast1)
arima_mase = MASE(df_training,df_testing,arima_fcast1,7)
                              

In [172]:
get_error_metrics("ARIMA", arima_mase, arima_smape, arima_rmse, arima_mae, arima_msmape)

# POOLED REGRESSION

In [173]:
pip install git+https://github.com/madrury/py-glm.git

In [174]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [175]:
def create_input_matrix(train_list,lag):
    
    embeded_train = np.asarray([]).reshape((0,lag+1))
    final_lags=np.asarray([]).reshape((0,lag))
    series_mean =[]
  #embeded_test = np.asarray([])
    for i in range(len(train_list)):
        train_list[i]=train_list[i].astype(np.float64)
    #test_list[i]=test_list[i].astype(np.float64)
    #mean=pd.concat((train_list[i],test_list[i])).mean()
        mean = train_list[i].mean()
        series_mean.append(mean)
        norm_train= train_list[i]/mean
    #norm_test = test_list[i]/mean
        embed=rolling_window(np.array(norm_train), lag+1)[:, ::-1]
        #print(embed)

        embeded_train = np.concatenate((embeded_train,embed),axis=0)   
    #final_lag = np.array(train_list[i][:lag:-1]).T
        final_lag = np.array(norm_train[-lag:][::-1])
        final_lag = final_lag.reshape((-1,lag))
        #print(final_lag.shape)
        final_lags = np.concatenate((final_lags,final_lag),axis=0)

    p4= pd.DataFrame(embeded_train)
    final_lags_pd = pd.DataFrame(final_lags)
    p4.columns = ['lag'+str(col_name) for col_name in p4.columns]
    final_lags_pd.columns = ['lag'+str(col_name+1) for col_name in final_lags_pd.columns]
    return p4, final_lags_pd, series_mean

In [176]:
def make_formula(df):
    formula = "lag0 ~ "
    lt =[]
    lt = df.columns
    for i in range(1,len(lt)):
        formula+=str(lt[i])
        if i!=len(lt)-1:
            formula+=" + "
        
    return formula 

In [177]:
def get_vals(lag, model, final_lags, forecast_horizon, series_means):
    pred_list = np.asarray([]).reshape((final_lags.shape[0],0))
    #print(pred_list.shape)
    for i in range(forecast_horizon):
        #print(final_lags) 
        new_pred = model.predict(final_lags)
        temp =new_pred.reshape((final_lags.shape[0],1))
        #print(new_pred)
        pred_list = np.concatenate((pred_list,temp),axis = 1)
        if i<forecast_horizon-1:
          
            final_lags = final_lags.iloc[:,:lag-1]
            final_lags =pd.concat((pd.DataFrame(new_pred),final_lags),axis = 1)
            final_lags.columns = ['lag'+str(i+1) for i in range(len(final_lags.columns))]
          
    #print(pred_list) 
    #s= np.array(series_means).reshape(pred_list.shape[0],pred_list.shape[1])
    return pred_list

In [178]:
from glm.glm import GLM
from glm.families import Gaussian, Bernoulli, Poisson, Exponential
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')
lag = 3
linear_model = GLM(family=Gaussian())

In [179]:
n_training_list = []
n_training_list.append(df_training)
n_testing_list = []
n_testing_list.append(df_testing)
matrix,final_lag,series_mean =  create_input_matrix(n_training_list,lag)


In [180]:
linear_model.fit(matrix, formula=make_formula(matrix))
y = get_vals(lag,linear_model,final_lag,n_testing_list[0].shape[0],series_mean)
for j in range(y.shape[1]):
        y[:,j] = y[:,j]*np.array(series_mean)

In [181]:
pl_smape = SMAPE(n_testing_list[0],y)
pl_rmse = RMSE(n_testing_list[0],y)
pl_mae = MAE(n_testing_list[0],y)
pl_msmape = msMAPE(n_testing_list[0],y)
pl_mase = MASE(n_training_list[0],n_testing_list[0],y,7)

In [182]:
get_error_metrics('Pooled Regression', pl_mase, pl_smape, pl_rmse, pl_mae, pl_msmape)

# ENSEMBLING 3 MODELS : THETA , ARIMA and POOLED REGRESSION

In [183]:
def getAverage(y_pred1, y_pred2, y_pred3):
    y_pred =  (np.array(y_pred1) + np.array(y_pred2) + np.array(y_pred3))/3
    return pd.Series(y_pred)

In [184]:
y1 = pd.Series(y.reshape(-1))

In [185]:
theta_fcast1.shape, y1.shape, arima_fcast1.shape

In [186]:
pred_avg = getAverage(theta_fcast1, y1, arima_fcast1)

In [187]:
pred_avg

In [188]:
ensemble_smape = SMAPE(df_testing,pred_avg)
ensemble_rmse = RMSE(df_testing,pred_avg)
ensemble_mae = MAE(df_testing,pred_avg)
ensemble_msmape = msMAPE(df_testing,pred_avg)
ensemble_mase = MASE(df_training,df_testing,pred_avg,7)

In [189]:
get_error_metrics("ENSEMBLING THETA , ARIMA and POOLED REGRESSION", ensemble_mase, ensemble_smape, ensemble_rmse, ensemble_mae, ensemble_msmape)